In [1]:
NB_SIM = 5

In [2]:
import sys

sys.path.append('..')

In [3]:
def rec_dict_print(a_dict, level=1):
    keys = a_dict.keys()
    keys.sort()
    for k in keys:
        if type(a_dict[k]) is dict:
            print "".join(["-" for i in range(level)]) + k
            rec_dict_print(a_dict[k], level + 2)
        else:
            print "".join(["-" for i in range(level)]) + k

In [4]:
%matplotlib inline
import dill

loaded_data = None

with open('precomputed_sims/data%i.pkl' % (NB_SIM), 'rb') as f:
    loaded_data = dill.load(f)

In [5]:
rec_dict_print(loaded_data)

-N
-credit
---bc_ids
---bc_subsets_indexes
---copula
---default_times
-underlyings


In [6]:
N = loaded_data["N"]

In [7]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1 / 360.
delta = 5 * step

maturity = 5.0

print "Maturity = %s years" % maturity

Maturity = 5.0 years


# Default times

In [8]:
c_ids = [17, 9, 29, 26, 50, 4, 5, 13, 64]

In [9]:
copula = loaded_data["credit"]["copula"]
default_times_mat = loaded_data["credit"]["default_times"]
c_subsets_indexes = loaded_data["credit"]["bc_subsets_indexes"]

In [10]:
import pandas as pd
import numpy as np

def get_default_times_sim(c_ids_, N_, default_times_mat_, copula_, subsets_indexes_):
    matrix_def_ = np.zeros((len(c_ids_), N_))
    
    for j_, c_id_ in enumerate(c_ids_):
        cp_subsets_indexes_ = copula_.get_indexes_including(c_id_)
        c_df_times_indexes_ = [ii_ for ii_, ind_ in enumerate(subsets_indexes_) if ind_ in cp_subsets_indexes_]
        
        c_df_times_mat_ = default_times_mat_[:, c_df_times_indexes_]
        c_df_times_ = c_df_times_mat_.min(axis=1)
        
        matrix_def_[j_] = c_df_times_
        
    matrix_def_[matrix_def_==1000.] = np.nan
        
    df_default_times_ = pd.DataFrame(matrix_def_, index=np.array(c_ids_))
    return df_default_times_

In [11]:
default_times = get_default_times_sim(c_ids, N, default_times_mat, copula, c_subsets_indexes)
default_times

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,0.336111,NaN,NaN,NaN,NaN,NaN,NaN,0.388889,...,1.808333,NaN,NaN,4.041667,NaN,NaN,NaN,NaN,NaN,NaN
26,0.941667,NaN,0.677778,NaN,0.686111,NaN,NaN,NaN,2.119444,1.347222,...,0.755556,NaN,0.875,1.172222,2.552778,0.469444,NaN,0.894444,0.702778,NaN
50,NaN,1.597222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.530556
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.638889,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,NaN,0.152778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.172222,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
import os

default_times_path = './res/sim%i/' % (NB_SIM)

path = os.path.join(default_times_path, 'df_times.csv')
default_times.to_csv(path)